목표
--------
* url을 각각의 변수로 나누어서 실습한다.
* erd를 보고 필요한 내용들을 파싱해본다.
* 파싱한 데이터를 db에 저장한다.

In [3]:
#필요한 라이브러리 import
import requests
import urllib.request
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
from pymysql import NULL
import pymysql
import re
import sys

In [50]:
conn = pymysql.connect(host="localhost", user='queant', password='queant1234', db = 'queant', charset='utf8mb4')
cur = conn.cursor()

In [31]:
#api활용할 url부분

#각 데이터 별 url 앞부분
url_deposit = ["https://finlife.fss.or.kr/finlifeapi/depositProductsSearch.xml?auth=", "https://finlife.fss.or.kr/finlifeapi/savingProductsSearch.xml?auth="]

#발급받은 키
api_key = "47c0e868fdb16333d47d0e385641c3c0"

#권역코드
code_front = "&topFinGrpNo="
code_bank = ["020000", "030300"]

#페이지
page_front = "&pageNo="
page_num = ["1", "2", "3", "4"]

* 아래 코드는 예,적금 상품의 가입방법을 파싱하는 코드
    * join_way 태그에 접근해서 데이터를 받는다.
    * 기존의 데이터는 스마트폰, 인터넷, 영업점 등으로 표현되어있다.
    * DB에는 공통코드를 통해 저장할 것이기 때문에 각각의 가입방법 존재유무를 파악해야한다.
    * Api를 볼때 가입방법은 총 영업점, 인터넷, 스마트폰, 전화, 기타로 총 5가지이다.
    * 각각의 문자열이 join_way 데이터에 있는지 확인하고 그 유무를 True, False로 배열에 저장할 것.

In [ ]:
for product_tag in data_xml[5]:
    join = product_tag[0].find("join_way").text
    print(join)
    join_parsing = ["영업점" in join, "인터넷" in join, "스마트폰" in join, "전화" in join, "기타" in join]
    print(join_parsing)
    print()

* 아래 코드는 예,적금 상품의 가입 나이 제한을 파싱하는 코드
    * 데이터들을 보았을때 일정한 형식이 없다.
    * 이를 어떻게 파싱할지 계획하는 것이 중요.

예외적인 문구들
- 만19세~만34세 개인 및 개인사업자
- 만 10세 이하 자녀를 둔 부, 모
- 1954년 이전출생자 및 비과세종합저축가입대상자(만64세이상/국가유공자/국민기초생활보장법에의한수급자/장애인/독립유공자과그유족또는가족/고엽제후유증환자 및 5.18민주화운동부상자)
- 만 20~30대만 가입가능 (나이제한)
- 1~19세


<구상 1>
1. 과정
    1. 단어들 사이의 공백을 모두 없애서 다 붙인다.
    2. 만, 세 단어가 들어가는 문장들만 추출
    3. "이상"이라는 단어가 들어가는 문장에서 "만oo세이상" 부분 문자열의 oo부분만 추출한다.
    4. 마찬가지로 "이하", "초과" 단어가 들어가는 문장에서 나이만 뽑아온다.
2. 문제점
    1. 만 13세~ 만 24세라는 조건에서는 추출 불가능
    2. ~이상 ~이하 문구에서 추출하면 나이의 최소는 잘 추출되지만 최대 부분은 '만 ~세이상 만~' 식으로 추출됨.

<구상 2>
1. 과정
    1. 단어들 사이의 공백을 모두 없애서 다 붙인다.
    2. 문자열에서 숫자들만 추출한다.
2. 문제점
    1. 문자열에 있는 숫자가 무조건 나이라는 보장이 없다.

결론
- 거의 대부분의 조건들이 나이를 표기할 때 만~세 형식으로 사용
- 모든 만~세 문자열의 사이 부분을 추출해서 배열에 저장
- 배열길이가 1이면 이상, 이하, 미만, 초과 등의 단어에 따라 max,min값 저장
- 배열길이가 1이 아니면 정렬 후 앞의 데이터를 min에 뒤에 데이터를 max에 저장

- 해결해야할 부분: 만 ~세 미만 만~세 이상 일 경우에는 미만에서 '만'이라는 단어를 판단해서 오류 발생

In [34]:
def max_min_join(join_member):
    min_age = None
    max_age = None
    join_member = join_member.replace(" ", "")
    if "만" in join_member and "세" in join_member:
        ages = re.findall('만(.+?)세', join_member)
        ages = list(map(int, ages))
        if len(ages) == 1:
            if "이상" in join_member:
                min_age = ages[0]
            elif "초과" in join_member:
                min_age = ages[0] + 1
            elif "이하" in join_member:
                max_age = ages[0]
            elif "미만" in join_member:
                max_age = ages[0] - 1
        else:
            ages.sort()
            min_age = int(ages[0])
            max_age = int(ages[1])
    return min_age, max_age

In [ ]:
for product_tag in data_xml[5]:
    join_deny = product_tag[0].find("join_deny").text
    join_member = product_tag[0].find("join_member").text
    min, max = max_min_join(join_member)
    print(join_member)
    print(min, max)
    print()

* 아래 코드는 가입 최소금액, 최대금액, 최대기간, 최소기간을 파싱할 코드
    * 가입 최소, 최대기간은 option 내용들로 활용할 예정

    문구들
    - 계약금액 1백만원이상
    - 가입금액 : 10만원 이상
    - 최소가입금액: 10만원 이상
    - 예금가입금액 10만원이상
    - 1인당 가입금액 : 10만원 이상
    - 계좌당 가입 최저한도 : 100만원
    - 가입한도: 5 백만원이상
    - 가입한도: 3백만원이상~3억원이하
    - 가입금액 : 1백만원이상 최고 50백만원(1인1계좌)
    - 가입금액 : 1천만원 이상 ~ 4천만원 이하
    - 최저 100만원 이상
    - 10만원이상 가입가능
    - 보호금융상품 1인당 최고 5천만원
    - 1.최소금액 : 10만원 이상  2.1인당 가입금액 한도 : 4,870만원
    - 1인당 가입한도:5천만원
    - 1계좌당가입한도:월50만원(최저1만원이상)

    예외적인 문구
    - 만65세이상비과세종합저축5,000만원가능
    - 예금자보호법에의해원리금기준5,000만원까지보호




    필수 문구: 계약금액, 최소금액, 한도, 가입금액, 최저, 이상, 최대, 이하

    파싱방법
    1. 필수 문구가 아닌것들은 모두 None값 반환
    2. 00만원, 00천원, 00억원 등등 숫자+~원 으로 구성된 문자열 부분을 파싱 후 costs 배열에 저장
    3. 글로 쓰여져 있는 가격을 숫자로 반환하는 함수 구성 후 costs값들 int형으로 변환
    4. 최대 최소값 추출
    5. 최대, 최소 값이 같다면 "이상", "이하" 등등 조건문으로 최대값인지 최소값인지 판단.

In [36]:
#string형태의 가격을 int형으로 변환
def change_cost(cost):
    multi = 1
    if "억원" in cost:
        multi = 100000000
    elif "천만원" in cost:
        multi = 10000000
    elif "백만원" in cost:
        multi = 1000000
    elif "만원" in cost:
        multi = 10000
    elif "천원" in cost:
        multi = 1000
    
    number = 0
    for ch in cost:
        if ch >= '0' and ch <= '9':
            number = number * 10 + int(ch)
    return number*multi

In [37]:
#최대 최소금액 추출
def max_min_cost(etc_note):
    min_cost = None
    max_cost = None
    etc_note = etc_note.replace(" ", "")
    etc_note = etc_note.replace("\n","")
    etc_note = etc_note.replace(",","")
    print(etc_note)
    tags = ["계약금액","최소금액","한도","가입금액","최저","이상","최대","이하","최고"]
    
    #기본적인 문구들이 포함되어있지 않으면 가격과 무관하므로 return 해준다.
    filter1 = 0
    for tag in tags:
        if tag in etc_note:
            filter1 = 1
           
    if filter1 == 0:
        return min_cost, max_cost
    
    flag = 0 #숫자가 등장하면 0이 됨
    costs = []
    costs_num = []
    cost = ""
    for i in etc_note:
        if i >= '0' and i <= '9' and flag == 0: #숫자가 처음 등장했을 경우
            flag = 1
            cost += i
            continue
        if flag == 1:
            if i >= '0' and i <= '9':
                cost += i
            elif i == '만' or i == '억' or i == '천' or i == '백' or i == '십':
                cost += i
            elif i == '원':
                cost += i
                flag = 0
                costs.append(cost)
                cost = ""
            else:
                flag = 0
                cost = ""
    for n in costs:
        costs_num.append(change_cost(n))
    if not costs_num:
        return min_cost, max_cost
    min_cost = min(costs_num)
    max_cost = max(costs_num)
    
    if min_cost == max_cost:
        if "이상" in etc_note:
            max_cost = None
        elif "이하" in etc_note or "미만" in etc_note:
            min_cost = None
        elif "한도":
            min_cost = None
    
    return min_cost, max_cost

In [ ]:
for product_tag in data_xml[5]:
    etc = product_tag[0].find("etc_note").text
    min_cost, max_cost = max_min_cost(etc)
    print(min_cost, max_cost)

* 아래 코드는 우대사항을 어떻게 파싱할지 구상하는 코드  
    우선적으로 어떤 우대조건들이 있나 파악

* 기획
    * 공통코드 table을 활용할 예정
    * 자연어적인 요소가 너무 많아서 해시태그 느낌으로 갈까 생각중
    * 특정 조건의 우대금리가 몇인지는 전체 조건을 읽어서 파악
    * 다만 상품을 검색할때 특정 우대조건이 있는지 판단하기는 가능

* 우대조건 사항
    * 첫 거래
    * 카드 가입
    * 재예치


* 파싱기획
    일단 문자열을 다 땡겨서 하나로 만들기는 불가능

    * 전용이란 단어 빼기

    * 비대면
    * 모바일
    * 급여
    * 연금
    * 공과금
    * 재예치
    * 입출금통장
    * 첫거래
    * 카드

* 특수문자
    * 특수문자중에 숫자(동그라미)부분 아스키코드값 9312~9321(1~10)




In [40]:
#특수문자 (숫자+동그라미)를 일반 숫자. 로 바꾸는 메서드
def spcl_number(st):
    for i in range(10):
        st = st.replace(chr(9312+i), chr(49+i)+'.')
    return st

In [39]:
#한줄로 되어있는 조건들 깔끔하게 파싱
def split_line_num(st):
    st = spcl_number(st)
    p = re.compile("[0-9][.][^0-9]")
    cnt = p.finditer(st)
    #print(cnt)
    if cnt:
        num = 0
        for i in cnt:
            index = i.start()
            st_front = st[:index+num]
            st_back = st[index+num:]
            if st_front == "":
                continue
            if st_front[-1] == '\n':
                continue
            st = st_front + '\n' + st_back
            num+=1

    return st

In [41]:
def filter_spcl(split_spcl):
    spcl_list = []
    for i in split_spcl:
        if i.isspace():
           continue
        if not i:
            continue
        spcl_list.append(i)
        
    return spcl_list

In [209]:
#몇 퍼센트인지 뽑아내는 메서드
def find_percent(tag):
    k = re.compile("[0-9][.][0-9]+[%]")
    per = k.search(tag)
    if per:
        rate = per.group()
        rate = rate.replace("%","")
        return float(rate)
    else:
        return None

In [163]:
# 앞부분 목록 리스트 나타내는 표시들 없애주는 메서드
def delete_number(tag):
    tag_front = tag[:10]
    tag_back = tag[10:]
    tag_front = tag_front.replace("-", "")
    tag_front = tag_front.replace("▶", "")
    tag_front = tag_front.replace("*", "")
    tag_front = tag_front.replace("-", "")
    p = re.compile("[0-9][.][^0-9]")
    is_num = p.search(tag)
    if is_num:
        idx = is_num.end()
        tag = tag_front[(idx-1):] + tag_back
    else:
        tag = tag_front + tag_back
    
    tag = tag.lstrip()
    return tag

In [193]:
def spcl_parsing(spcl):
    spcl = spcl.replace("  :", " ")
    spcl = spcl.replace(":  "," ")
    spcl = spcl.replace("  0", " :0")
    spcl = spcl.replace("  ", "\n")
    spcl = split_line_num(spcl)
    
    split_spcl = spcl.split("\n")
    split_spcl = filter_spcl(split_spcl)

    special_list = ["비대면","모바일","급여","연금","공과금","재예치","입출금통장","첫거래","카드","인터넷"]

    flag = 0
    all_percent = 0
    data_list = []
    for tag in split_spcl:
        #첫번째줄에 전체 항목에 대한 퍼센트가 있을경우
        if flag == 0 and tag == split_spcl[0]:
            if "항목" in tag:
                flag = 1
                new_tag = tag[tag.find("항목"):]
                all_percent = find_percent(new_tag)
                continue
        
        if "최고" in tag or "최대" in tag:
            continue
                            
        percent = find_percent(tag)
        if percent == None:
            if flag == 1:
                percent = all_percent
            else:
                continue
        specials=[]
        for tag_list in special_list:
            if tag_list in tag:
                specials.append(tag_list)
        if not specials:
            specials.append("기타")
        
        tag = delete_number(tag)
        data = (specials, percent, tag)
        data_list.append(data)
    return data_list   


In [188]:
url = url_deposit[0] + api_key + code_front + code_bank[0] + page_front + page_num[0]
data_str = urllib.request.urlopen(url).read().decode('euc-kr')
data_xml = ET.fromstring(data_str)

In [216]:
for product_tag in data_xml[5]:
    special = product_tag[0].find("spcl_cnd").text
    #print("<원래 내용>")
    #print(special)
    #print()
    #print("<파싱 내용>")
    lists = spcl_parsing(special)
    print(lists)

[]
[(['기타'], 0.1, 'WON 통장에서 출금 후 연결 가입 시 연 0.1%P'), (['기타'], 0.1, '만기해지 시 오픈뱅킹 타행계좌 등록 연 0.1%P')]
[]
[(['기타'], 1.0, '신규 직전년도 말일 기준 우리은행 계좌를 보유하지 않은 경우 연 1.0%p'), (['기타'], 0.4, '직전 1년간 우리은행 예.적금 상품을 보유하지 않은 경우 연 0.4%p'), (['기타'], 0.4, '오픈뱅킹 서비스 연 0.4%p')]
[]
[(['기타'], 0.2, '목돈굴리기예금 최초 가입시 : 0.2%'), (['카드'], 0.2, '최근 1개월 이내 신용(체크)카드 신규 발급 시 : 0.2%'), (['인터넷'], 0.2, '최근 1개월 이내 인터넷.폰.스마트뱅킹 가입시 : 0.2%'), (['인터넷'], 0.05, '인터넷/스마트뱅킹을 통해 가입시 우대 : 0.05%')]
[(['연금'], 0.1, '연금실적우대 : 0.1%'), (['카드'], 0.1, '신용(체크)카드 보유 : 0.1%'), (['기타'], 0.1, '거래실적우대 : 0.1%'), (['기타'], 0.1, 'DGB행복파트너적금 동시보유 : 0.1%'), (['인터넷'], 0.05, '인터넷/스마트뱅킹을 통해 가입시 : 0.05%')]
[(['기타'], 0.1, '총 수신 평잔실적 or 첫만남플러스 통장 보유시 : 0.1%'), (['기타'], 0.1, '주택청약상품보유 : 0.1%'), (['기타'], 0.1, '신규일 "DGB함께적금" 동시 가입 후 만기일까지 보유 : 0.1%'), (['기타'], 0.1, '대구은행 오픈뱅킹서비스에 다른 은행 계좌 등록시 : 0.1%'), (['인터넷'], 0.05, '인터넷/스마트뱅킹을 통해 가입 시 :0.05%')]
[]
[(['기타'], 0.1, '승용차요일제 참여고객 : 0.1%'), (['기타'], 0.05, '저공해 자동차 보유고객 : 0.05%'), (['기타'], 0.05, '탄소포인트제 참여고객 : 0.05%')

* 아래 코드는 옵션 부분들을 어떻게 파싱할지 구상하는 코드
    * 필요한 변수
        * base_rate: 기본 금리
        * save_trm: 저축 기간
        * rate_type: 금리 유형명 (단리, 복리)
        * rsrv_type: 저축방식 (자유적립, 정액적립)
    * 공통코드 활용 안하기 때문에 직접 저장하면 된다.

In [ ]:
for product_tag in data_xml[5]:
    print("new prdt")
    for option_tag in product_tag[1]:
        rate = option_tag.find("intr_rate").text
        save_term = option_tag.find("save_trm").text
        rate_type = option_tag.find("intr_rate_type_nm").text
        rsrv_type = option_tag.find("rsrv_type_nm").text
        print(rate, save_term, rate_type, rsrv_type)